In [ ]:
#python3 t2_process.py --tof ts1_16x320_inv --ntile 16 --tsize 80

In [1]:
import os, gc, subprocess, time, sys, shutil, argparse

import scipy
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from tqdm import tqdm
#from tqdm.notebook import tqdm
import skimage.io
from skimage.transform import resize, rescale
from math import sqrt
import statistics

sys.path.append("..")

In [2]:
import PIL.Image
#PIL.Image.MAX_IMAGE_PIXELS = 400000000
PIL.Image.MAX_IMAGE_PIXELS = None

In [3]:
#%matplotlib inline

In [4]:
from panda_bvv_config import *

## 1. Inputs ##

In [5]:
Zoom = 1
input_folder = train_path
input_mask_folder = mask_size1


In [ ]:
ap = argparse.ArgumentParser()
ap.add_argument('--tof', help="folder name to save tile", dest = 'tof',\
                type=str)

args = vars(ap.parse_args())

In [ ]:
#data-from folder:
to_folder = args["tof"]

In [ ]:
new_proc_folder = os.path.join(base_path, to_folder)

In [7]:
# wrapping inputs:
if not os.path.exists(new_proc_folder):
    print("[INFO] 'creating {}' directory".format(new_proc_folder))
    os.makedirs(new_proc_folder)


## 2. Standard functions ##

In [9]:
#wrapper for image processing function
def process_all_images(proc_func):
    
    def wrapper(zoom = Zoom,
                input_biopsy_folder = input_folder,
                output_data_path = new_proc_folder,
                df_name = train_labels,
                tif_file = False,
                with_mask = False,
                input_mask_folder = input_mask_folder,
                **kw):
        print(input_biopsy_folder)
        print(new_proc_folder)
        with tqdm(total=df_name.shape[0]) as pbar:
            for  i, row in enumerate(tqdm(df_name.iterrows())):
                img_num = row[1]['image_id']
                provider = row[1]['data_provider']                         
                mask = None
                if tif_file:
                    try:
                        biopsy = skimage.io.MultiImage(os.path.join(input_biopsy_folder,\
                                                                    img_num + '.tiff'))[zoom]               
                    except:
                        print('Failed to read tiff:', img_num)                        
                else:
                    try:
                        biopsy = skimage.io.imread(os.path.join(input_biopsy_folder, img_num + '.png'))
                        shape = biopsy.shape
                        if shape[0]<shape[1]: 
                            biopsy = np.rot90(biopsy)
                            shape = biopsy.shape 
                    except:
                        print(f'can not proceed with {img_num}')   
                if with_mask:
                    try:
                        mask = skimage.io.imread(os.path.join(input_mask_folder, img_num + '_mask.png'))
                        shape = mask.shape
                        if shape[0]<shape[1]: 
                            mask = np.rot90(mask)
                            shape = mask.shape 
                    except:
                        print('Failed to process mask:', img_num)
                try:
                    data_new = proc_func(biopsy, mask, provider, **kw)
                    cv2.imwrite(os.path.join(output_data_path, img_num + '.png'), data_new)
                except Exception as ee:
                    print('Processing mistake:\n', ee, '\n', img_num)            
                try:
                    del biopsy, mask, data_new, mask_new
                except:
                    pass
                pbar.update(1)
            gc.collect()
        return
    
    return wrapper

## 3. Processing functions ##

In [14]:
@process_all_images
def tiff_to_png_size1(img, mask, provider, **kw):
    return img

## 4. Process input files for training purpose ##

In [ ]:
tiff_to_png_size1(tif_file = True,
                  df_name = train_labels)

import os
module_name = 't2_extract_size1'

os.system('jupyter nbconvert --to python ' + module_name + '.ipynb')
with open(module_name + '.py', 'r') as f:
    lines = f.readlines()
with open(module_name + '.py', 'w') as f:
    for line in lines:
        if 'nbconvert --to python' in line:
            break
        else:
            f.write(line)